Охуительная идея для структуры голосования:

по строкам - два метода

по колонкам - разные ядра

в ячейках - количество выигранных раундов (1 раунд - 1 граф)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.spectral_clustering import SpectralClustering
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels

n_clusters = 4
generator = GraphGenerator(100, n_clusters, 0.2, 0.05)

for kernel_class in get_all_kernels():
    ward_results = {}
    spectral_results = {}
    all_ward_scores = []
    all_spectral_scores = []
    for i in range(0, 10):
        graph = generator.generate_graph()
        nodes = graph.nodes
        edges = graph.edges
        ward_scores = []
        spectral_scores = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            ward_prediction = Ward(n_clusters).fit_predict(K)
            ward_score = adjusted_rand_score(nodes, ward_prediction)
            ward_scores.append(ward_score)
            spectral_prediction = SpectralClustering(n_clusters).fit_predict(K)
            spectral_score = adjusted_rand_score(nodes, spectral_prediction)
            spectral_scores.append(spectral_score)
        all_ward_scores.append(ward_scores)
        all_spectral_scores.append(spectral_scores)
    all_ward_scores = [list(i) for i in zip(*all_ward_scores)]
    all_spectral_scores = [list(i) for i in zip(*all_spectral_scores)]
    resulting_ward_score = [np.mean(x) for x in all_ward_scores]
    resulting_spectral_score = [np.mean(x) for x in all_spectral_scores]
    ward_results[kernel_class.name] = resulting_ward_score
    spectral_results[kernel_class.name] = resulting_spectral_score
    
    with open('src/results/my/my_graphs/ward' + kernel_class.name + '.txt', 'w') as file:
        file.write(json.dumps(ward_results))
    with open('src/results/my/my_graphs/k-means' + kernel_class.name + '.txt', 'w') as file:
        file.write(json.dumps(spectral_results))
        
    plt.figure(figsize=(10,10))
    print('Ward ', kernel_class.name, 'best result: ', np.max(resulting_ward_score))
    print('K-means ', kernel_class.name, 'best result: ', np.max(resulting_spectral_score))
    for kernel_name, resulting_score in ward_results.items():
        print(kernel_name, 'best result: ', np.max(resulting_score))
        plt.plot(kernel_class.default_params, resulting_score, label='Ward')
    for kernel_name, resulting_score in spectral_results.items():
        print(kernel_name, 'best result: ', np.max(resulting_score))
        plt.plot(kernel_class.default_params, resulting_score, label='K-means')
    # plt.plot(kernel_class.default_params, resulting_spectral_score, label='K-means')
    plt.title(kernel_class.name + ', n_clusters=' + str(n_clusters))
    plt.legend()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.spectral_clustering import SpectralClustering
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels


generator = GraphGenerator(200, 4, 0.2, 0.05)
n_clusters = 4

for kernel_class in get_all_kernels():
    results = {}
    all_scores = []
    for i in range(0, 50):
        graph = generator.generate_graph()
        nodes = graph.nodes
        edges = graph.edges
        scores = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = SpectralClustering(n_clusters).fit_predict(K)
            score = adjusted_rand_score(nodes, prediction)
            scores.append(score)
        all_scores.append(scores)
    all_scores = [list(i) for i in zip(*all_scores)]
    resulting_score = [np.mean(x) for x in all_scores]
    results[kernel_class.name] = resulting_score
    
    with open('src/results/my/' + kernel_class.name + '.txt', 'w') as file:
        file.write(json.dumps(results))
        
    plt.figure(figsize=(10,10))
    for kernel_name, resulting_score in results.items():
        print(kernel_name, 'best result: ', np.max(resulting_score))
        plt.plot(kernel_class.default_params, resulting_score, label=kernel_name)
    plt.title('K-means ' + kernel_class.name + ', n_clusters=' + str(n_clusters))
    plt.legend()
    plt.show()

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = news

for data in all_data:
    graph, info = data
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = Ward(info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = str(max(scores_1)) + ' ' + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + str(max(scores_1)) + ' ' + str(max(scores_2)))
    print('data ' + info['name'] + ' finished!')
    with open('src/results/my/news/' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(results))

news_2cl_1 PlainWalk 0.046538240459 0.131073947536


news_2cl_1 LogPlainWalk 0.765036841281 0.666211167588


news_2cl_1 Communicability 0.0398139851986 0.137056806879


news_2cl_1 LogCommunicability 0.0398139851986 0.137056806879


news_2cl_1 Forest 0.0 0.0157732781716


...

news_3cl_1 LogPlainWalk 0.698476902125 0.626289453208


news_3cl_1 Communicability 0.396134440162 0.527740021915


news_3cl_1 LogCommunicability 0.396134440162 0.527740021915


news_3cl_1 Forest 1.1167038899e-05 0.0223206287314


news_3cl_1 LogForest 1.1167038899e-05 0.0223206287314


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = [football, polbooks, zachary, polblogs]

for data in all_data:
    graph, info = data
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = Ward(info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = str(max(scores_1)) + ' ' + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + str(max(scores_1)) + ' ' + str(max(scores_2)))
    print('data ' + info['name'] + ' finished!')
    with open('src/results/my/others/' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(results))

football PlainWalk 0.906334157557 0.930810884185


football LogPlainWalk 0.896650009791 0.92419578687


football Communicability 0.909377088152 0.935845790439


football LogCommunicability 0.909377088152 0.935845790439


football Forest 0.896650009791 0.92419578687


...

zachary Forest 0.153463105478 0.272495191494


zachary LogForest 0.153463105478 0.272495191494


zachary Heat 1.0 1.0


zachary LogHeat 1.0 1.0
data zachary finished!


KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.kernel_kmeans import KernelKMeans
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

all_data = [football, polbooks, zachary]

for data in all_data:
    graph, info = data
    print('data ' + info['name'] + ' for Ward started!')
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            prediction = Ward(info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = 'ARI=' + str(max(scores_1)) + ' NMI=' \
                                     + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + results[kernel_class.name])
    print('data ' + info['name'] + 'for Ward' + ' finished!')
    with open('src/results/my/datasets/' + 'Ward_' + info['name'] + '.txt', 'w') as file:
        file.write(json.dumps(results))

data football for Ward started!


football PlainWalk ARI=0.906334157557 NMI=0.930810884185


KeyboardInterrupt: 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from src.clustering.kernel_kmeans import KernelKMeans
from src.clustering.ward import Ward
from src.graph.graph_generator import GraphGenerator
from src.kernel.kernel import get_all_kernels
from src.dataset.dataset import football, polbooks, polblogs, zachary, news

# all_data = [football, polbooks, zachary]
all_data = [zachary]

for data in all_data:
    graph, info = data
    print('data ' + info['name'] + 'for K-means' + ' started!')
    print('n_clusters = ' + str(info['k']))
    results = {}
    for kernel_class in get_all_kernels():
        results[kernel_class.name] = {}
        nodes = graph.nodes
        edges = graph.edges
        scores_1 = []
        scores_2 = []
        kernel = kernel_class(edges)
        for K in kernel.get_Ks():
            print(kernel.name, 'K = ', K)
            prediction = KernelKMeans(n_clusters=info['k']).fit_predict(K)
            score_1 = adjusted_rand_score(nodes, prediction)
            scores_1.append(score_1)
            score_2 = normalized_mutual_info_score(nodes, prediction)
            scores_2.append(score_2)
        results[kernel_class.name] = 'ARI=' + str(max(scores_1)) + ' NMI=' + str(max(scores_2))
        print(info['name'] + ' ' + kernel_class.name + ' ' + results[kernel_class.name])
    print('data ' + info['name'] + 'for K-means' + ' finished!')
    # with open('src/results/my/datasets/' + 'K_means_' + info['name'] + '.txt', 'w') as file:
    #     file.write(json.dumps(results))

data zacharyfor K-means started!
n_clusters = 2
PlainWalk K =  [[  1.00090044e+00   7.83710774e-03   7.72856260e-03 ...,   7.44551597e-03
    1.71301685e-04   2.27553613e-04]
 [  7.83710774e-03   1.00050796e+00   7.66981848e-03 ...,   6.08439124e-05
    1.15466509e-04   1.71666115e-04]
 [  7.72856260e-03   7.66981848e-03   1.00056255e+00 ...,   1.71538858e-04
    7.50306750e-03   3.41406714e-04]
 ..., 
 [  7.44551597e-03   6.08439124e-05   1.71538858e-04 ...,   1.00033446e+00
    7.50250750e-03   7.55989853e-03]
 [  1.71301685e-04   1.15466509e-04   7.50306750e-03 ...,   7.50250750e-03
    1.00067491e+00   8.00441185e-03]
 [  2.27553613e-04   1.71666115e-04   3.41406714e-04 ...,   7.55989853e-03
    8.00441185e-03   1.00095343e+00]]
PlainWalk K =  [[  1.00204661e+00   1.20774963e-02   1.18354084e-02 ...,   1.11900272e-02
    3.92646796e-04   5.20610135e-04]
 [  1.20774963e-02   1.00115602e+00   1.16989737e-02 ...,   1.43613784e-04
    2.66099294e-04   3.93796261e-04]
 [  1.18354084e-02

LogPlainWalk K =  [[  9.00031110e-04  -4.84888542e+00  -4.86283238e+00 ...,  -4.90014331e+00
   -8.67208432e+00  -8.38812469e+00]
 [ -4.84888542e+00   5.07827176e-04  -4.87046233e+00 ...,  -9.70719879e+00
   -9.06653003e+00  -8.66995916e+00]
 [ -4.86283238e+00  -4.87046233e+00   5.62393941e-04 ...,  -8.67070074e+00
   -4.89244334e+00  -7.98243608e+00]
 ..., 
 [ -4.90014331e+00  -9.70719879e+00  -8.67070074e+00 ...,   3.34399357e-04
   -4.89251798e+00  -4.88489751e+00]
 [ -8.67208432e+00  -9.06653003e+00  -4.89244334e+00 ...,  -4.89251798e+00
    6.74678183e-04  -4.82776241e+00]
 [ -8.38812469e+00  -8.66995916e+00  -7.98243608e+00 ...,  -4.88489751e+00
   -4.82776241e+00   9.52971164e-04]]
LogPlainWalk K =  [[  2.04452183e-03  -4.41641137e+00  -4.43665953e+00 ...,  -4.49273233e+00
   -7.84260009e+00  -7.56050910e+00]
 [ -4.41641137e+00   1.15535078e-03  -4.44825416e+00 ...,  -8.84838292e+00
   -8.23164103e+00  -7.83967689e+00]
 [ -4.43665953e+00  -4.44825416e+00   1.27755469e-03 ...,  -

LogPlainWalk K =  [[ 1.82215514  1.3674896   1.50352388 ...,  0.96833384  1.37420573
   1.56804573]
 [ 1.3674896   1.36114391  1.22864492 ...,  0.62328771  1.08566058
   1.2811174 ]
 [ 1.50352388  1.22864492  1.59894778 ...,  0.82480935  1.32237629
   1.48935156]
 ..., 
 [ 0.96833384  0.62328771  0.82480935 ...,  0.91155774  0.88058075
   1.06432998]
 [ 1.37420573  1.08566058  1.32237629 ...,  0.88058075  1.58837742
   1.5628366 ]
 [ 1.56804573  1.2811174   1.48935156 ...,  1.06432998  1.5628366
   1.89552801]]
zachary LogPlainWalk ARI=1.0 NMI=1.0
Communicability K =  [[  1.00565841e+00   2.88575761e-02   2.81798039e-02 ...,   2.63993852e-02
    1.07891362e-03   1.43230764e-03]
 [  2.88575761e-02   1.00319372e+00   2.78080199e-02 ...,   3.87256912e-04
    7.28595941e-04   1.08164866e-03]
 [  2.81798039e-02   2.78080199e-02   1.00353488e+00 ...,   1.08082675e-03
    2.67623239e-02   2.14932988e-03]
 ..., 
 [  2.63993852e-02   3.87256912e-04   1.08082675e-03 ...,   1.00209831e+00
    2.6


Communicability K =  [[ 90969.20859148  67864.22075377  79396.06670979 ...,  46364.4209015
   73595.75353406  89206.42811461]
 [ 67864.22075377  50693.15305056  59374.99559717 ...,  34708.70638579
   55195.84228729  66895.38811763]
 [ 79396.06670979  59374.99559717  70293.45496527 ...,  41809.23593192
   67121.99102782  81258.39597219]
 ..., 
 [ 46364.4209015   34708.70638579  41809.23593192 ...,  25602.19931751
   41661.23244485  50352.67830276]
 [ 73595.75353406  55195.84228729  67121.99102782 ...,  41661.23244485
   68410.08567777  82598.69189521]
 [ 89206.42811461  66895.38811763  81258.39597219 ...,  50352.67830276
   82598.69189521  99748.80301647]]
Communicability K =  [[  988539.60257289   738509.8355823    871650.6248123  ...,
    516299.37879609   826444.02006143  1000798.35494302]
 [  738509.8355823    551928.94147185   651879.95175318 ...,
    386466.08611674   619116.55336555   749677.84051569]
 [  871650.6248123    651879.95175318   774361.18906623 ...,
    463341.671291

 K =  [[ 5.9265824   3.64715027  3.52509631 ...,  1.72924892  1.69029195
   2.11952489]
 [ 3.64715027  3.78458028  2.86215799 ...,  0.92096245  1.27858766
   1.63803807]
 [ 3.52509631  2.86215799  4.12119144 ...,  1.37809241  2.33975802
   2.60020391]
 ..., 
 [ 1.72924892  0.92096245  1.37809241 ...,  2.44993022  2.02892078
   2.41763198]
 [ 1.69029195  1.27858766  2.33975802 ...,  2.02892078  4.65766562
   4.45754434]
 [ 2.11952489  1.63803807  2.60020391 ...,  2.41763198  4.45754434
   6.11997498]]
LogCommunicability K =  [[ 8.03544515  5.15040757  5.04703551 ...,  2.45255095  2.58989803
   3.23546081]
 [ 5.15040757  4.96426893  4.04694647 ...,  1.40888001  1.96362834
   2.49462281]
 [ 5.04703551  4.04694647  5.48635605 ...,  2.04859132  3.379778
   3.86261119]
 ..., 
 [ 2.45255095  1.40888001  2.04859132 ...,  3.027297    2.85721668
   3.41935079]
 [ 2.58989803  1.96362834  3.379778   ...,  2.85721668  6.22229401
   6.29980725]
 [ 3.23546081  2.49462281  3.86261119 ...,  3.41935079 

Forest K =  [[ 0.27795254  0.04000535  0.03280928 ...,  0.02758722  0.00736559
   0.0086425 ]
 [ 0.04000535  0.40246738  0.0426212  ...,  0.00602669  0.00795342
   0.01001371]
 [ 0.03280928  0.0426212   0.37479883 ...,  0.01224471  0.02956299
   0.01617069]
 ..., 
 [ 0.02758722  0.00602669  0.01224471 ...,  0.49459136  0.03673411
   0.03294624]
 [ 0.00736559  0.00795342  0.02956299 ...,  0.03673411  0.33681982
   0.03929812]
 [ 0.0086425   0.01001371  0.01617069 ...,  0.03294624  0.03929812
   0.26614015]]
Forest K =  [[ 0.2573354   0.04070654  0.0331381  ...,  0.02748494  0.00790485
   0.0092345 ]
 [ 0.04070654  0.37671312  0.04321277 ...,  0.00671592  0.00867158
   0.01081911]
 [ 0.0331381   0.04321277  0.34942891 ...,  0.01330046  0.02967638
   0.01724321]
 ..., 
 [ 0.02748494  0.00671592  0.01330046 ...,  0.4665368   0.03703667
   0.0333155 ]
 [ 0.00790485  0.00867158  0.02967638 ...,  0.03703667  0.31333679
   0.04004942]
 [ 0.0092345   0.01081911  0.01724321 ...,  0.0333155   0.0

LogForest K =  [[ 0.32755408  0.03805432  0.03177955 ...,  0.02752273  0.00621526
   0.00736074]
 [ 0.03805432  0.46144077  0.04078601 ...,  0.0046566   0.00645481
   0.00828464]
 [ 0.03177955  0.04078601  0.43337323 ...,  0.00998597  0.02907562
   0.01375122]
 ..., 
 [ 0.02752273  0.0046566   0.00998597 ...,  0.55647069  0.03566453
   0.03185959]
 [ 0.00621526  0.00645481  0.02907562 ...,  0.03566453  0.39200665
   0.03728574]
 [ 0.00736074  0.00828464  0.01375122 ...,  0.03185959  0.03728574
   0.31446701]]
LogForest K =  [[ 0.30117655  0.03913263  0.0323665  ...,  0.02761057  0.00680406
   0.00801986]
 [ 0.03913263  0.43058445  0.04182641 ...,  0.00533968  0.00721497
   0.00916987]
 [ 0.0323665   0.04182641  0.4026433  ...,  0.01113995  0.02937285
   0.01500937]
 ..., 
 [ 0.02761057  0.00533968  0.01113995 ...,  0.52449767  0.03629039
   0.03247118]
 [ 0.00680406  0.00721497  0.02937285 ...,  0.03629039  0.36288453
   0.03838757]
 [ 0.00801986  0.00916987  0.01500937 ...,  0.0324711

Heat K =  [[ 0.05753885  0.04980421  0.03712821 ...,  0.02688332  0.01231047
   0.01425007]
 [ 0.04980421  0.12974784  0.05278209 ...,  0.01107176  0.01436276
   0.0179242 ]
 [ 0.03712821  0.05278209  0.10389821 ...,  0.02260091  0.03000444
   0.02838293]
 ..., 
 [ 0.02688332  0.01107176  0.02260091 ...,  0.21679146  0.04161954
   0.03711789]
 [ 0.01231047  0.01436276  0.03000444 ...,  0.04161954  0.08319156
   0.049011  ]
 [ 0.01425007  0.0179242   0.02838293 ...,  0.03711789  0.049011
   0.05332652]]
Heat K =  [[ 0.05418812  0.04919253  0.036363   ...,  0.0254703   0.01271394
   0.01467698]
 [ 0.04919253  0.11449004  0.05145268 ...,  0.01213082  0.01522253
   0.01877869]
 [ 0.036363    0.05145268  0.09066246 ...,  0.02393754  0.02902267
   0.02918979]
 ..., 
 [ 0.0254703   0.01213082  0.02393754 ...,  0.18994046  0.0400795
   0.03636544]
 [ 0.01271394  0.01522253  0.02902267 ...,  0.0400795   0.07493232
   0.04859767]
 [ 0.01467698  0.01877869  0.02918979 ...,  0.03636544  0.04859767


LogHeat K =  [[ 0.03334263  0.03172283  0.02923327 ...,  0.02624755  0.02518695
   0.02563396]
 [ 0.03172283  0.03220837  0.0302483  ...,  0.02769412  0.02740288
   0.02773078]
 [ 0.02923327  0.0302483   0.03023843 ...,  0.02982446  0.02987326
   0.02995412]
 ..., 
 [ 0.02624755  0.02769412  0.02982446 ...,  0.03347467  0.03246436
   0.03224382]
 [ 0.02518695  0.02740288  0.02987326 ...,  0.03246436  0.03440559
   0.03381155]
 [ 0.02563396  0.02773078  0.02995412 ...,  0.03224382  0.03381155
   0.03337081]]
LogHeat K =  [[ 0.03215778  0.03085801  0.02916684 ...,  0.02714069  0.02639811
   0.02670097]
 [ 0.03085801  0.03088308  0.0297642  ...,  0.02830413  0.02807409
   0.02826518]
 [ 0.02916684  0.0297642   0.02982457 ...,  0.02975098  0.02983474
   0.02984805]
 ..., 
 [ 0.02714069  0.02830413  0.02975098 ...,  0.03194629  0.03176297
   0.03157662]
 [ 0.02639811  0.02807409  0.02983474 ...,  0.03176297  0.03292649
   0.03255491]
 [ 0.02670097  0.02826518  0.02984805 ...,  0.03157662  

In [9]:
print((404+298+154+249+71+120) * 2)
print(-42-91-41-139-31-399-553)
print(12*11/2*50)

2592
-1296
3300.0
